# MedScan (Medical Scanner)

<p align="center">
  <img src="./images/cover.png" alt="Cover">
</p>


## Introduzione

Si vuole costruire un sistema che sia in grado di elaborare in real time dei referti medici. In particolare, si vogliono estrarre dai documenti informazioni quali lo stato delle strutture anatomiche esaminate, la diagnosi medica e altre informazioni legate alla salute del paziente.

## Data flow

<center><img src="./images/schema-tecnologie.png"></center>

## Data source

I dati analizzati sono stati forniti da un medico.

Sono delle diagnosi formulate a seguito di esami ecografici alle regioni inguino-scrotale maschili.

<center><img src="./images/dati.png"></center>

## Data Ingestion

<img src="./images/logstash-cover.png" width="30%">

Logstash è un progetto open-source per l'analisi dei log in tempo reale. E' utilizzato per accumulare i testi dei referti. <br>

Logstash esegue uno script che legge tutti i file e in real time li spedisce a Kafka che fungerà da storage.

### File di configurazione

```python 
input {
  exec {
    command => "python3 /ingest.py"
    codec => multiline {
      pattern => "---"
      negate => true
      what => "next"
    }
    interval => 3600 # 3600 s == 1 h
  }
}

output {
  stdout{}
  kafka {
    bootstrap_servers => "10.0.100.23:9092"
    codec => json
    topic_id => ner
  }
}
```

## Data streaming
<img src="./images/kafka-logo.png" >

### Cosa è Kafka? 

Secondo RedHat: "*Apache Kafka è una piattaforma per il data streaming distribuita che  permette di pubblicare, sottoscrivere, archiviare ed elaborare flussi di record in tempo reale. È progettata per gestire flussi di dati  provenienti da più fonti distribuendoli a più consumatori. In breve,  consente di spostare grandi quantità di dati da un punto qualsiasi a un  altro nello stesso momento.*"

E' stato creato un topic dove vengono conservati tutti i dati e da cui leggerà Spark.

<center><img src="./images/kafka-ui.png" height="50%"></center>

<center><img src="./images/meme_kafka_streaming.jpg" width="50%" height="30%"></center>

## Data Processing

<img src="./images/apache_spark_cover.png" width="20%">

Apache Spark™ is a multi-language engine for executing data engineering, data science, and machine learning on single-node machines or clusters.
<br>
Nello specifico, utilizziamo **Spark Structured Streaming**, che ci permette di lavorare su dati in tempo reale raggruppati all'interno di un'astrazione ad alto livello, il dataframe. Un dataframe è come una tabella contenente i dati e su cui possono essere svolte delle operazioni.<br>
Attraverso Spark structured streaming otteniamo un dataframe (in streaming) contenente i dati trasmessi in tempo reale da Kafka.

Lo schema di un messaggio Kafka:

```python
# Define the schema of the JSON data from Kafka
json_schema = StructType() \
    .add("tags", ArrayType(StringType())) \
    .add("message", StringType()) \
    .add("process", StructType()
         .add("exit_code", StringType())
         .add("command_line", StringType())) \
    .add("@version", StringType()) \
    .add("@timestamp", StringType()) \
    .add("host", StructType()
         .add("name", StringType())) \
    .add("event", StructType()
         .add("original", StringType()))
```

### ChatGPT
<img src="./images/chatgpt-logo.png" >


ChatGPT è il Large Language Model più popolare al mondo è uno tra i più performanti.
E' stato allenato con tutti i dati che sono presenti su Internet. Svolge una quantità vastissima di tasks. Alcuni esempi:

- generare nuovi articoli, e-mail o poesie seguendo un determinato stile che può essere quello di uno scrittore

- Generare codice

-  Spiegare concetti complessi in modo semplice

- Riassumere testi

- Correggere errori sintattici 



Grazie all'estrema versalità del modello, è possibile inserire diversi prompts ovvero la ricerca si può focalizzare su determinate caratteristiche piuttosto che altre sulla base dell'esigenze dell'utente; ad esempio un neurologo potrebbe essere interessato a estrarre informazioni diverse rispetto a quelle che cercherebbe un dermatologo.

<center><img src="./images/meme-spark-chatgpt.png"></center>

Per ottenere degli ottimi risultati, è fondamentale saper chiedere esattamente cosa si vuole al modello. I promtps non devono lasciare spazio ad ambiguità, così il modello saprà esattamente cosa cercare.<br>

* SYSTEM_PROMPT è il messaggio con il quale definiamo chi è e che cosa dovrà fare il modello.
* GUIDELINES_PROMPT è il messaggio più importante : specifica che cosa si deve cercare, e come la risposta deve essere formattata.  


```python
SYSTEM_PROMPT = '''
                Sei un intelligente sistema che estrae delle caratteristiche da un testo.\
                Ti fornirò quali caratteristiche devi estrarre, il testo da cui estrarle e il formato di output.
                '''
USER_PROMPT_1 = "Hai chiaro il tuo ruolo?"

ASSISTANT_PROMPT_1 = "Certo, sono pronto ad aiutarti con il tuo compito. Ti prego di fornirmi le informazioni necessarie per iniziare."

GUIDELINES_PROMPT = (
"Formato Output:\n"
"{{\n"
    "\"Emiscroto destro\" : {{\n"
        "\"Testicolo\" : {{\n"
            "\"Sede anatomica\" : \"\",\n"
            "\"Volume\" : \"\",\n"
            "\"Dimensioni\" : \"\",\n"
            "\"Ecostruttura : \"\"\n"
        "}},\n"
        "\"Epididimo\" : {{\n"
            "\"Aspetto ecografico\" : \"\"\n"
        "}},\n"
        "\"Canale inguinale\" : \"\"\n"
    "}},\n"
    "\n"
    "\"Emiscroto sinistro\" : {{\n"
        "\"Testicolo\" : {{\n"
            "\"Sede anatomica\" : \"\",\n"
            "\"Volume\" : \"\",\n"
            "\"Dimensioni\" : \"\",\n"
            "\"Ecostruttura : \"\"\n"
        "}},\n"
        "\"Epididimo\" : {{\n"
            "\"Aspetto ecografico\" : \"\"\n"
        "}},\n"
        "\"Canale inguinale\" : \"\"\n"
    "}}\n"
"}}\n"
"Spiegazione caratteristiche :\n"
"\"Sede anatomica\" può assumere solo i valori \"Fisiologica\" o \"Non fisiologica\" o \"\"\n"
"\"Volume\" può assumere solo i valori \"Normali per età\" o \"Non normali per età\" o \"\"\n"
"\"Dimensioni\" può assumere solo i valori \"Normali per età\" o \"Non normali per età\" o \"\"\n"
"\"Ecostruttura\" può assumere solo i valori \"Omogenea\" o \"Disomogenea\" o \"\"\n"
"\"Aspetto ecografico\" può assumere solo i valori \"Normale\" o \"Anormale\" o \"\"\n"
"\"Canale inguinale\" può assumere solo i valori \"Normale\" o \"Anormale\" o \"\"\n"
"Testo : {}\n"
"Output : "
)
```

<center><img src="./images/meme-prompt.png"></center>

Per poter effettuare chiamate alle API di OpenAI è necessario creare un account Azure, richiedere l'accesso alle API e importare l'endopoint e la secret key. 

```bash 
echo 'export AZURE_OPENAI_ENDPOINT=<Your OpenAI Endpoint>' >> ~/.bashrc
echo 'export AZURE_OPENAI_KEY=<Your OpenAI Secret Key>' >> ~/.bashrc
source ~/.bashrc
```

Il seguente pezzo di codice setta le variabili neccessarie per effettuare chiamate a GPT-4.

```python
def setOpenAIConf():
    openai.api_type = "azure"
    openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
    openai.api_version = "2023-03-15-preview"
    openai.api_key = os.getenv("AZURE_OPENAI_KEY")
```

Questa funzione fa una chiamata al modello, passando come parametri, tra gli altri, i prompts che abbiamo definito sopra.

```python
def requestToChatGPT(user_prompt):
   setOpenAIConf()
   return openai.ChatCompletion.create(
      engine="healthcare-features-extractor",
      messages = [
        {"role":"system","content":SYSTEM_PROMPT},
        {"role":"user","content":USER_PROMPT_1},
        {"role": "assistant", "content": ASSISTANT_PROMPT_1},
        {"role": "user", "content": user_prompt}
        ],
      temperature=0,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
```

Il pezzo di codice seguente sanitizza l'output ricevuto da ChatGPT-4 per evitare che ci siano JSON non validi e invia il risultato ,sotto forma di file JSON, a Elasticsearch.

```python
def sendToEs(batch_df: DataFrame, batch_id: int):
  for row in batch_df.rdd.collect():
    user_prompt = GUIDELINES_PROMPT.format(row['message'])
    
    response = requestToChatGPT(user_prompt)
    response = response['choices'][0]['message']['content']
    if response[0] != '{':
        continue

    try:
        doc = json.loads(response.rsplit('}', 1)[0] + '}')
        doc["Emiscroto destro"]["Testicolo"]["Volume"] = float(doc["Emiscroto destro"]["Testicolo"]["Volume"])
        doc["Emiscroto sinistro"]["Testicolo"]["Volume"] = float(doc["Emiscroto sinistro"]["Testicolo"]["Volume"])
    except (json.JSONDecodeError, ValueError):
        continue
    
    doc["Timestamp"] = row["timestamp"]

    global idx
    idx = idx + 1
    resp = es.index(index=ELASTIC_INDEX, id=idx, document=doc)
    
```

## Data indexing

<img src="./images/elasticsearch_cover.png" width="30%">

Elasticsearch è un motore di ricerca e analisi distribuito (open source) per tutti i tipi di dati, inclusi testuali, numerici, geospaziali,  strutturati e non strutturati. 

// Foto dell'indice ner_idx

## Data Visualization

Kibana è l'anello mancante...

<img src="./images/kibana_cover.png" width=30%>

È un’interfaccia web estensibile per la presentazione visiva dei dati raccolti. Insieme ad Elasticsearch e allo strumento di elaborazione Logstash forma il cosiddetto stack ELK.

Creiamo delle dashboard che ci permettono di visualizzare l'andamento dei dati, la predizione e altre preziose informazioni, il tutto in tempo reale. Vediamo qualche grafico d'esempio!

<center><img src="./images/dashboards.png"></center>